# Delivered Cost Modeling Approach
This approach is based on [Hogland et al. 2018](https://www.mdpi.com/2220-9964/7/4/156) and uses [Raster Tools](https://github.com/UM-RMRS/raster_tools) to perform spatial analyses. By default roads, streams, and water bodies are downloaded from [OpenStreetMap](https://www.openstreetmap.org/about) and elevation data is downloaded from [3DEP](https://www.usgs.gov/3d-elevation-program). To get started, use the interactive web map to zoom into a area of interest. Next:

1. Draw a polygon on the map that spatially identifies the extent of the study area.
2. Draw a point on the map that spatially locates conversion facility(s) for biomass delivery.
3. Specify various machine rates.
4. Click the run button. Delivered cost surfaces will be added to the Map. 

In [ ]:
import delvCost
import ipyleaflet
import ipywidgets
import localtileserver
import ipyfilechooser
import base64
import os

if('LOCALTILESERVER_CLIENT_PREFIX' in os.environ.keys()):
    os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = f"{os.environ['JUPYTERHUB_SERVICE_PREFIX']}/proxy/{{port}}"

lc = ipyleaflet.LayersControl()
lc.position='topright'
dr=ipyleaflet.DrawControl()
dr.position='topleft'


bmaps=ipyleaflet.basemaps

m=ipyleaflet.Map(controls=[lc,dr],layout=ipywidgets.Layout(height='90%'),scroll_wheel_zoom=True,center=(41,-119),zoom=5)

tly1=ipyleaflet.basemap_to_tiles(bmaps.Esri.WorldImagery)
tly1.base = False
tly1.name = 'World Imagery'

tly2=ipyleaflet.ImageService(
    url=r'https://lfps.usgs.gov/arcgis/rest/services/Landfire_LF230/US_230EVT/ImageServer',
    format='jpgpng',
    attribution='USDA USFS',
    name = 'Landfire EVT',
)


tly3=ipyleaflet.ImageService(
    url=r'https://apps.fs.usda.gov/fsgisx03/rest/services/wo_spf_fam/Nat_BurnProbability/ImageServer',
    format='jpgpng',
    attribution='USDA FAM',
    name = 'Burn Probability'
)

tly4=ipyleaflet.ImageService(
    url=r'https://apps.fs.usda.gov/fsgisx03/rest/services/wo_spf_fam/Potential_Control_Location/ImageServer',
    format='jpgpng',
    attribution='USDA FAM',
    name='PCL',
)
tly5=ipyleaflet.ImageService(
    url=r'https://apps.fs.usda.gov/fsgisx03/rest/services/wo_spf_fam/SnagHazard_eDart/ImageServer',
    format='jpgpng',
    attribution='USDA FAM',
    name='Snag Hazard eDart',
)
tly6=ipyleaflet.ImageService(
    url=r'https://apps.fs.usda.gov/fsgisx03/rest/services/wo_spf_fam/Suppression_Difficulty_Index_80/ImageServer',
    format='jpgpng',
    attribution='USDA FAM',
    name='SDI 80',

)

baselyrDic = {
    'World Imagery':tly1,
    'Landfire EVT': tly2,
    'Burn Probability': tly3,
    'PCL': tly4,
    'Snag Hazard eDart':tly5,
    'SDI 80':tly6,
}

def add_remove_layer(b):
    nm = b['owner'].description
    lyr = baselyrDic[nm]
    if b['new']:
        m.add(lyr)
    else:
        m.remove(lyr)


#widgets
style = {'description_width': 'initial'}

#optional background layers
l1_o=ipywidgets.widgets.Checkbox(value=False,description='World Imagery', disabled=False, indent=False)
l1_o.observe(add_remove_layer,names='value')
l2_o=ipywidgets.widgets.Checkbox(value=False,description='Landfire EVT', disabled=False, indent=False)
l2_o.observe(add_remove_layer,names='value')
l3_o=ipywidgets.widgets.Checkbox(value=False,description='Burn Probability', disabled=False, indent=False)
l3_o.observe(add_remove_layer,names='value')
l4_o=ipywidgets.widgets.Checkbox(value=False,description='PCL', disabled=False, indent=False)
l4_o.observe(add_remove_layer,names='value')
l5_o=ipywidgets.widgets.Checkbox(value=False,description='Snag Hazard eDart', disabled=False, indent=False)
l5_o.observe(add_remove_layer,names='value')
l6_o=ipywidgets.widgets.Checkbox(value=False,description='SDI 80', disabled=False, indent=False)
l6_o.observe(add_remove_layer,names='value')

thb=ipywidgets.HBox([l1_o,l2_o,l3_o,l4_o,l5_o,l6_o])  

lbl=ipywidgets.Label(value='Add/Remove Background Layers')
lbl.style.font_size="20px"
thb=ipywidgets.VBox([lbl,thb])
#main widgets
tr_s=ipywidgets.widgets.FloatSlider(value=1.5,min=1.0,max=5.0,step=0.1,description='RT Skidder Speed (MPH):',style=style)
cb_s=ipywidgets.widgets.FloatSlider(value=2.0,min=1.0,max=5.0,step=0.1,description='Skyline Speed (MPH):',style=style)

tr_d=ipywidgets.widgets.FloatSlider(value=165,min=150,max=200,step=1,description='RT Skidder ($/hour):',style=style)
cb_d=ipywidgets.widgets.FloatSlider(value=400,min=300,max=500,step=1,description='Skyline ($/hour):',style=style)
fb_d=ipywidgets.widgets.FloatSlider(value=15,min=5,max=100,step=1,description='Fellerbuncher ($/CCF):',style=style)
hf_d=ipywidgets.widgets.FloatSlider(value=27,min=5,max=100,step=1,description='Hand Felling ($/CCF):',style=style)
pr_d=ipywidgets.widgets.FloatSlider(value=56,min=5,max=200,step=1,description='Processing ($/CCF):',style=style)
ha_d=ipywidgets.widgets.FloatSlider(value=98,min=70,max=200,step=1,description='Hauling ($/hour):',style=style)
ht_d=ipywidgets.widgets.FloatSlider(value=2470,min=0,max=4000,step=5,description='Hand Treatment ($/acre):',style=style)
pf_d=ipywidgets.widgets.FloatSlider(value=2470,min=0,max=4000,step=5,description='Prescribed Fire ($/acre):',style=style)

tr_p=ipywidgets.widgets.FloatSlider(value=1.25,min=0.5,max=30.0,step=0.1,description='RT Skidder Payload (CCF):',style=style)
cb_p=ipywidgets.widgets.FloatSlider(value=1.04,min=0.5,max=30.0,step=0.1,description='Skyline Payload (CCF):',style=style)
lt_p=ipywidgets.widgets.FloatSlider(value=12.25,min=5.0,max=60,step=0.1,description='Log Truck Payload (CCF):',style=style)

cb_o=ipywidgets.widgets.Checkbox(value=False,description='Create Optional surfaces', disabled=False, indent=False)

#optional widgets
roads_name=ipyfilechooser.FileChooser(filter_pattern=['*.shp'],title='User Specified Roads')
barriers_name=ipyfilechooser.FileChooser(filter_pattern=['*.shp'],title='User Specified Barriers')

prg_wdg=ipywidgets.widgets.IntProgress(value=0,min=0,max=12,description='Progress',style={'bar_color':'green'},orientation='horizontal')
out_wdg=ipywidgets.widgets.Output(layout={'border': '1px solid black'})

def get_local_tile_service(path, name='',palette='Greys',nodata=-999999,vmin=None, vmax=None):
    client1 = localtileserver.TileClient(path)
    t1 = localtileserver.get_leaflet_tile_layer(client1, band=[1], name=name, palette=palette,nodata=nodata,vmin=vmin,vmax=vmax)
    m.add(t1)
    return t1

def create_download_link(filename, title = "Click here to download: "):  
    fl = open(filename, "rb")
    data = fl.read()
    b64 = base64.b64encode(data)
    payload = b64.decode()
    fl.close()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank" style="color:blue">{title}</a>'
    html = html.format(payload=payload,title=title+f' {filename}',filename=filename)
    return html

@out_wdg.capture()
def delv_cost(b):
    prg_wdg.value=0
    out_wdg.clear_output()

    delvCost.sk_r=tr_s.value
    delvCost.cb_r=cb_s.value
    delvCost.sk_d=tr_d.value
    delvCost.cb_d=cb_d.value
    delvCost.fb_d=fb_d.value
    delvCost.hf_d=hf_d.value
    delvCost.pr_d=pr_d.value
    delvCost.lt_d=ha_d.value
    delvCost.pf_d=pf_d.value
    delvCost.sk_p=tr_p.value
    delvCost.cb_p=cb_p.value
    delvCost.lt_p=lt_p.value
    delvCost.cb_o=cb_o.value
    delvCost.pbar=prg_wdg

    #optional
    delvCost.lyr_roads_path=roads_name.value
    delvCost.lyr_barriers_path=barriers_name.value

    #check values
    pntlst=[]
    polylst=[]
    for d in dr.data:
        geo=d['geometry']
        t=geo['type']
        if(t=='Point'):pntlst.append(geo['coordinates'])
        if(t=='Polygon'):polylst.append(geo['coordinates'][0])
    
    if((len(pntlst)<1) | (len(polylst)<1)):
        print("You Must Create a Study Area afore clicking the Run button")
        return
    else:
        delvCost.study_area_coords = polylst
        delvCost.saw_coords = pntlst
        outdic=delvCost.run()
        for k in outdic.keys():
            vl = outdic[k]
            get_local_tile_service(path=vl,name=k,palette='RdYlGn_r')
            html=create_download_link(vl)
            wb.children=tuple(list(wb.children) +[ipywidgets.HTML(html)])
        
        return

  
btn=ipywidgets.widgets.Button(description='Run')
btn.on_click(delv_cost)


wb=ipywidgets.VBox([ipywidgets.widgets.Label(""),ipywidgets.widgets.Label(""),tr_s,cb_s,tr_d,cb_d,fb_d,hf_d,pr_d,ha_d,ht_d,pf_d,tr_p,
                   cb_p,lt_p,cb_o,roads_name,barriers_name,btn],layout=ipywidgets.Layout(width='30%'))

rb=ipywidgets.VBox([thb,m],layout=ipywidgets.Layout(width='70%',height='800px'))

mb=ipywidgets.HBox([wb,rb])

ipywidgets.VBox([mb,prg_wdg,out_wdg])